# Maximum projections from z-stacks

This notebook computes the maximum projection of confocal z-stack images. It then saves the images in a separate folder with or without a scalebar (define below). Image matrices are saved to a dictionary and continuously linked to their sample#, channel#, condition, and filename.

#### Name images as follows: 
X_samplecondition_sample_channel , where "X" is optional and can be filled as desired by the user (do not use spaces).  
- "Samplecondition" can be a string of text separated by "-", e.g. "GCaMP-30C-6d_sample_channel" (Do not use spaces or "_" within the condition, else part of it will be cut out in the dictionary.)  
- "Sample" and "channel" can be a simple number or a description, e.g.: "samplecondition_1_1" or "samplecondition_Brain1_DAPI". (Do not use spaces or "_" within the sample or channel.)  

#### Click "Run all" after defining the parameters below.

Written by Laura Luebbert, 20th of August 2020.  

Modified on: / 

# Define the parameters:

### Define the directory containing the tif files:

In [ ]:
data_dir = "/Users/"

### Define microscope and magnification (to get interpixel distance) OR define interpixel distance directly if different microscope/magnification used:

In [ ]:
# Define microscope used ("Lois", "LSM800" or "other")
microscope = "Lois"

# Define magnification ("20" or "other")
magnification = "20"

# Uncomment (remove the #) and define interpixel distance if different microscope/magnification used:
# interpixel_distance = 

length_units = "µm"

### Scale bar options:

In [ ]:
# Turn scale bar on/off (define as True or False)
scale_bar = True

# Set scalebar color ("white" or "black")
scale_bar_color = "white"

# Define the desired scale bar size and width in length unit.
scale_bar_length = 100
scale_bar_width = 5

# Define position of scale bar:
# Distance from top (Distance in % of total length of image)
y_pos = 95
# Distance of rightmost end of scale bar from left end of image (Distance in % of total length of image)
x_pos = 95

### Saving options:

In [ ]:
# Directory where maximum projection image folder will be created.
saving_dir = data_dir

# Set the file format the images should be saved as:
file_format = "tif"

### Define colors/LUTs of each channel for maximum projections:

In [ ]:
# Define dictionary with the channel#/channel-name (exactly as defined in the filename) coupled to "green", "red", or "blue"
# e.g. "1":"green" or "DAPI":"blue"  
color_dict = {"GFP":"green", "RFP":"red", "DAPI":"blue"}

### Filter options for maximum projections:

In [ ]:
# Despeckling using median filter (skimage.filters.rank.median) (define as True or False)
despeckle = True
despeckle_parameter = 3

# Contrast is optimized using the skimage.exposure.equalize_adapthist function (define as True or False)
optimize_contrast = True

___

___

In [ ]:
# %load_ext blackcellmagic
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Tools to read in the image files and filenames
import glob
import os
import re 

# Calculation and data frame tools
import numpy as np
import pandas as pd

# Image processing tools
import skimage
import skimage.io
import skimage.morphology

# Tools to create new folders
from pathlib import Path

# Tools to save dictionaries
import pickle

# To make a copy of a dictionary
import copy

# Plotting tools
import bokeh
import bokeh_catplot
bokeh.io.output_notebook()

___

# Get the interpixel distance

In [ ]:
if microscope == "LSM800":
    if magnification == "20":
        interpixel_distance = 0.3119629
        
if microscope == "Lois":
    if magnification == "20":
        interpixel_distance = 0.690534

___

# Load in the data

In [ ]:
# Glob string for images (loads all .tif files)
im_glob = os.path.join(data_dir, "*.tif")

# Get list of files in directory
im_list = sorted(glob.glob(im_glob))

# Let's look at the first 10 entries
im_list[:10]

Create a nested dictionary with information about each sample coupled to the z-stack image matrix, as well as space for matrices added later:

In [ ]:
dicts = {}

for i in range(len(im_list)):
    # Get sample condition
    condition = im_list[i].split("/")[-1].split("_")[-3]
    
    # Make sure we do not overwrite a previous dictionary entry    
    if dicts.get(condition) == None:
        dicts[condition] = {}
    
    # Get sample number
    sample = im_list[i].split("/")[-1].split("_")[-2]
    
    # Make sure we do not overwrite a previous dictionary entry
    if dicts.get(condition, {}).get(sample) == None:
        dicts[condition][sample] = {}
    
    # Get channel number    
    channel = im_list[i].split("/")[-1].split("_")[-1].split(".")[0]
    
    # Add empty arrays to dictionary to populate with images later
    dicts[condition][sample][channel] = {
        "matrix_orig": [],      # Original image z-stack matrix
        "matrix_max": [],       # Image matrix of maximum projection image 
        "matrix_max_8": [],     # Image matrix of maximum projection as 8-bit image 
        "matrix_max_8_SB": [],  # Image matrix of maximum projection as 8-bit image with scale bar 
        "filename": [],         # The original filename, just in case
    }    
    
    # Populate dictionary with original image matrix
    dicts[condition][sample][channel]["matrix_orig"] = skimage.io.imread(im_list[i])
    
    # Populate dictionary with original filename
    dicts[condition][sample][channel]["filename"] = im_list[i].split("/")[-1].split(".")[-2]

___

# Compute maximum projections
Find the maximum pixel value across the frames for each channel for each image:

In [ ]:
selem = skimage.morphology.square(despeckle_parameter)

# Loop through every image in the dictionary
for k1_condition, v1_sample in dicts.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            
            image = np.array(im["matrix_orig"])

            # Create a matrix of zeros in the same size as our image (x-pixels, y-pixels) to fill color channels we do not need
            zeros = np.zeros((image.shape[1], image.shape[2]))

            # np.max with axis=0 returns the maximum of each column (each row equals to a frame)
            max_channel = image.max(axis=0)

            # Image filter options
            if despeckle == True:
                max_channel = skimage.filters.rank.median(max_channel, selem)   

            if optimize_contrast == True:  
                max_channel = skimage.exposure.equalize_adapthist(max_channel)

            # Populate maximum projections dictionary with the max projection and name of the image
            # The color/LUT is based on its axis in the array (red, green or blue):
            if color_dict[k3_channel] == "red":
                dicts[k1_condition][k2_sample][k3_channel]["matrix_max"] = np.dstack((max_channel, zeros, zeros))

            if color_dict[k3_channel] == "green":
                dicts[k1_condition][k2_sample][k3_channel]["matrix_max"] = np.dstack((zeros, max_channel, zeros))

            if color_dict[k3_channel] == "blue":
                dicts[k1_condition][k2_sample][k3_channel]["matrix_max"] = np.dstack((zeros, zeros, max_channel))

___

# Save maximum projections

#### Scale down images:

To save the images using skimage, we need to scale them down to 8 bit.

In [ ]:
# Scale down images to 8 bits

for k1_condition, v1_sample in dicts.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            # Linearly scale image down to 8-bit.
            image = (im["matrix_max"] / im["matrix_max"].max()) * 255

            # Change list to array and change type to 8-bit array.
            image = np.array(image)
            image = image.astype(np.uint8)

            dicts[k1_condition][k2_sample][k3_channel]["matrix_max_8"] = image

### Display the last maximum projection using skimage:

In [ ]:
skimage.io.imshow(dicts[k1_condition][k2_sample][k3_channel]["matrix_max_8"])

#### Burn in scale bars:

Scale bar is burned into image by changing the pixel value to 1000 (white scale bar) or 0 (black scale bar) in scale bar area defined in the corresponding "parameters" cell above:

In [ ]:
if scale_bar == True:

    scalebar = 1 / interpixel_distance * scale_bar_length
    scalebar_width = 1 / interpixel_distance * scale_bar_width

    if scale_bar_color == "white":
        for k1_condition, v1_sample in dicts.items():
            for k2_sample, v2_channel in v1_sample.items():
                for k3_channel, im in v2_channel.items():
                    y_value = int((im["matrix_max_8"].shape[0]/100)*y_pos)
                    x_value = int((im["matrix_max_8"].shape[1]/100)*x_pos) 

                    im["matrix_max_8"][y_value : y_value + int(scalebar_width), x_value - int(scalebar) : x_value] = 255

                    # Populate dict with 8-bit images containing a scale bar
                    dicts[k1_condition][k2_sample][k3_channel]["matrix_max_8_SB"] = im["matrix_max_8"]

    elif scale_bar_color == "black":
        for k1_condition, v1_sample in dicts.items():
            for k2_sample, v2_channel in v1_sample.items():
                for k3_channel, im in v2_channel.items():
                    y_pos = int((im["matrix_max_8"].shape[0]/100)*y_pos)
                    x_pos = int((im["matrix_max_8"].shape[1]/100)*x_pos)

                    im["matrix_max_8"][y_pos : y_pos + int(scalebar_width), x_pos - int(scalebar) : x_pos] = 0

                    # Populate dict with 8-bit images containing a scale bar
                    dicts[k1_condition][k2_sample][k3_channel]["matrix_max_8_SB"] = im["matrix_max_8"]
            
else:
    for k1_condition, v1_sample in dicts.items():
        for k2_sample, v2_channel in v1_sample.items():
            for k3_channel, im in v2_channel.items():
            
                dicts[k1_condition][k2_sample][k3_channel]["matrix_max_8_SB"] = im["matrix_max_8"]

Display one image with scale bar:

In [ ]:
skimage.io.imshow(dicts[k1_condition][k2_sample][k3_channel]["matrix_max_8_SB"])

### Create max projection folder:

Create folder in saving directory to save maximum projections to (unless the folder already exists):

In [ ]:
path = ("{}/{}_max_projections").format(saving_dir, im_list[0].split("/")[-2])
Path(path).mkdir(parents=True, exist_ok=True)

### Save all images with a scale bar:

In [ ]:
for k1_condition, v1_sample in dicts.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            skimage.io.imsave(
                ("{}/{}_max.{}").format(path, im["filename"], file_format),
                im["matrix_max_8_SB"],
                plugin=None,
                check_contrast=True,
            )

### Pickle image data dictionary:

In [ ]:
# Use pickle to save dictionaries

# The advantage of HIGHEST_PROTOCOL is that files get smaller. This makes unpickling sometimes much faster.
with open(
    ("{}/{}.pickle").format(data_dir, im_list[0].split("/")[-2]), "wb"
) as handle:
    pickle.dump(dicts, handle, protocol=pickle.HIGHEST_PROTOCOL)

___

# Computing environment

In [ ]:
%load_ext watermark

%watermark -v -p re,numpy,pandas,skimage,bokeh,bokeh_catplot,jupyterlab